Ablauf:
Fiel "AQM_import_API"
1. Import Data via API from Refinitiv to Jupiter Notebook
    - 10 Commodities 10 Years
    - Download monthly data for at least 10 asset prices and at least 10 macroeconomic variables, across countries

2. Create DataFrame, check Data and clean Data if needed:
    - N/A
    - Sprünge?
    - Format?
3. Save clean Data in SQL Lite in DB


Zuerst den Code für 1 Kombination schreiben. Danach ein Loop generieren.

File "AQM_Analyse":
1. Import the clean Data from SQL DB into Notebook
2. Prepare Data for Analysis
3. Create Dataframe to save the Results
4. Create Loop, which executes multivariate OLS regressions using all combinations of 1 and 10 explanatory macroeconomic variables 
5. Save the Results from the Loop into the "Result"-Dataframe
    For each Regression calculate:
        Autocorrelation
        heteroskedasticity
        multicollinearity
6. Export Results to SQL Lite into Table "Results"

Todo:
- Direkt von API in Dataframe und danach in SQL ohne Excel
- Excel workaround sauber fertig machen

In [18]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


from datetime import datetime
import time
import datetime

import eikon as ek
creds = 'f5a341b614d945b9b93a71fe53348ba119d11e99'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
#API Key Laurin:
ek.set_app_key(creds)


In [19]:
#Define the Future Contracts
Futures = ['LCOc1',     #ICE Europe Brent Crude 
           'CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]

In [20]:
data_all = pd.DataFrame()
try:
    ek.set_app_key('f5a341b614d945b9b93a71fe53348ba119d11e99')
    itemName=Futures
    dateStart="2012-01-01"
    dateEnd="2022-01-01"

    #Create Loop to inport all Futures:
    for y in range(0,len(Futures)):
        data_input = ek.get_timeseries(Futures[y],fields=["Close"], start_date = dateStart , 
                                    end_date = dateEnd , interval='daily', corax = 'adjusted' )
        data_input['Instrument'] = Futures[y]
        data_all = data_all.append(data_input)
except ek.EikonError as ex:
    print('Eikon Error')
    print(ex)
except Exception as ex:
    print(ex)

data_all

,CLOSE,Instrument
Date,,
2012-01-03,112.13,LCOc1
2012-01-04,113.70,LCOc1
2012-01-05,112.74,LCOc1
2012-01-06,113.06,LCOc1
2012-01-09,112.45,LCOc1
...,...,...
2021-12-27,139.40,OJc1
2021-12-28,140.30,OJc1
2021-12-29,136.70,OJc1


In [21]:
#Unpivot the Data:
data_all = data_all.pivot(columns='Instrument', values='CLOSE')
data_all = data_all.reset_index()
data_all['Date'] = pd.to_datetime(data_all['Date']).dt.date
data_all.to_excel('Data_Refinitiv.xlsx')
liste_SQL = data_all.values.tolist()
data_all.head()

Instrument,Date,CLc1,Cc1,GCc1,LCOc1,LCc1,NGLNMc1,NGc1,OJc1,PAc1,PLc1,SIc1,Wc1
0,2012-01-03,102.96,658.5,1599.7,112.13,121.550,52.75,2.993,173.50,662.95,1427.3,29.533,657.00
1,2012-01-04,103.22,658.5,1611.9,113.70,121.625,53.09,3.096,184.75,653.00,1423.1,29.063,650.00
2,2012-01-05,101.81,643.5,1619.4,112.74,120.950,52.95,2.980,182.25,643.85,1414.5,29.265,629.25
3,2012-01-06,101.56,643.5,1616.1,113.06,120.325,52.87,3.062,182.00,613.45,1404.5,28.653,624.75
4,2012-01-09,101.31,652.0,1607.5,112.45,119.950,54.84,3.011,193.95,617.30,1427.5,28.749,641.75


In [ ]:
data_all.plot(figsize=(15,30), subplots=True);

In [ ]:
data_all.plot(figsize=(17,10));

In [ ]:
#calculate Daily Log Return:
data_all

Save Data in SQL Database (DB)

In [22]:
#alternative when API not working or to work offline: import the excel file
data_all = pd.read_excel('Data_Refinitiv.xlsx')
data_all = data_all.set_index('Date')
data_all = data_all.drop(['Unnamed: 0'], axis=1)
data_all = data_all.reset_index().rename_axis(None, axis=1)
data_all['Date'] = pd.to_datetime(data_all['Date'], format='%Y-%m-%d').dt.date
liste_SQL = data_all.values.tolist()
data_all


KeyError: "['CREATED_ON'] not found in axis"

In [ ]:
#Create a DB in SQL Lite
#insert the clean Data in the SQL DB: "SQL_DB" into the Table "REFENITIV_DATA":

In [23]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

2.6.0


In [24]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS REFINITIV_DATA;''')

In [25]:
 
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''CREATE TABLE REFINITIV_DATA (
    DATE DATETIME,
	"LCOc1" TEXT,
    "CLc1" TEXT,
    "NGLNMc1" TEXT,
    "NGc1" TEXT,
    "GCc1" TEXT,
    "SIc1" TEXT,
    "PLc1" TEXT,
    "PAc1" TEXT,
    "Wc1" TEXT,
    "Cc1" TEXT,
    "LCc1" TEXT,
    "OJc1" TEXT
);''')



In [26]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
#Add all Data at once:
c = conn.cursor()
c.executemany("INSERT INTO REFINITIV_DATA(DATE, LCOc1, CLc1, NGLNMc1, NGc1, GCc1 ,SIc1 ,PLc1 ,PAc1 ,Wc1 ,Cc1 ,LCc1 ,OJc1) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", liste_SQL)    
conn.commit()



In [27]:
#Select Data from SQL DB:   
c = conn.cursor()
c.execute("SELECT * FROM REFINITIV_DATA")
new_data=c.fetchall()
ref_data = pd.DataFrame(new_data,columns=['DATE', 'LCOc1', 'CLc1', 'NGLNMc1', 'NGc1', 'GCc1' ,'SIc1' ,'PLc1' ,'PAc1' ,'Wc1' ,'Cc1' ,'LCc1' ,'OJc1'])
ref_data = ref_data.set_index('DATE')
ref_data

,LCOc1,CLc1,NGLNMc1,NGc1,GCc1,SIc1,PLc1,PAc1,Wc1,Cc1,LCc1,OJc1
DATE,,,,,,,,,,,,
2012-01-03,102.96,658.5,1599.7,112.13,121.55,52.75,2.993,173.5,662.95,1427.3,29.533,657.0
2012-01-04,103.22,658.5,1611.9,113.7,121.625,53.09,3.096,184.75,653.0,1423.1,29.063,650.0
2012-01-05,101.81,643.5,1619.4,112.74,120.95,52.95,2.98,182.25,643.85,1414.5,29.265,629.25
2012-01-06,101.56,643.5,1616.1,113.06,120.325,52.87,3.062,182.0,613.45,1404.5,28.653,624.75
2012-01-09,101.31,652.0,1607.5,112.45,119.95,54.84,3.011,193.95,617.3,1427.5,28.749,641.75
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,75.57,614.75,1808.1,78.6,137.3,None,4.06,139.4,1944.3,968.9,22.979,804.0
2021-12-28,75.98,604.75,1810.2,78.94,138.475,None,4.055,140.3,1998.4,979.1,23.111,783.5
2021-12-29,76.56,605.5,1805.1,79.23,139.4,233.18,4.024,136.7,1984.8,968.2,22.848,787.75
